In [1]:
import pandas as pd
import numpy as np

import os
dirpath = os.getcwd()
dirpath

'C:\\Users\\agupta\\Downloads\\Analytics vidhya\\WNS_hackathon'

In [2]:
train = pd.read_csv(dirpath + '/train_NA17Sgz/train.csv')
view_log = pd.read_csv(dirpath + '/train_NA17Sgz/view_log.csv')
item_data = pd.read_csv(dirpath + '/train_NA17Sgz/item_data.csv')
test = pd.read_csv(dirpath + '/test_aq1FGdB/test.csv')

In [3]:
# converting string to datetime format
train['impression_time'] = pd.to_datetime(train.impression_time)
view_log['server_time'] = pd.to_datetime(view_log.server_time)
test['impression_time'] = pd.to_datetime(test.impression_time)

In [5]:
# extracting hour and weekday from time 
train['Hour'] = train.impression_time.dt.hour
train['Weekday'] = train.impression_time.dt.day_name()

test['Hour'] = test.impression_time.dt.hour
test['Weekday'] = test.impression_time.dt.day_name()


In [1]:
train.head(), test.head()

NameError: name 'train' is not defined

In [ ]:
print('Creating historic CTRs')
# create historic ctr
col_list = ['weekday', 'hour', 'app_code', 'is_4G', 'os_version']
for col in col_list:
    df_temp = df_train.groupby([col])\
                      .apply(lambda x: pd.Series([sum(x['is_click'])/len(x)], index=['click_rate']))\
                      .reset_index().rename(columns={'click_rate':col+'_ctr'})
    df_train = pd.merge(left=df_train, right=df_temp, how='left', on=[col])
    df_test = pd.merge(left=df_test, right=df_temp, how='left', on=[col])

In [8]:
df = pd.read_csv(dirpath + '/inputs/df_model.csv')

In [9]:
df.head()

,user_id,impression_id,impression_time,is_click,weekday_ctr,hour_ctr,app_code_ctr,is_4G_ctr,os_version_ctr,cnt_visit_l1d,...,cnt_visit_category_1_2_l1d,cnt_visit_category_1_2_l3d,cnt_visit_category_1_2_l7d,cnt_visit_category_1_2_l14d,cnt_visit_category_1_2_l30d,cnt_uniq_item_l1d,cnt_uniq_item_l3d,cnt_uniq_item_l7d,cnt_uniq_item_l14d,cnt_uniq_item_l30d
0,87862,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,0.0,0.045868,0.045731,0.146835,0.046258,0.049290,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,63410,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,1.0,0.045868,0.045731,0.061611,0.044752,0.041651,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0
2,71748,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,0.0,0.045868,0.045731,0.049822,0.044752,0.051762,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,69209,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,0.0,0.045868,0.045731,0.123369,0.044752,0.041651,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,62873,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,0.0,0.045868,0.045731,0.050202,0.046258,0.041651,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [10]:
df.cnt_visit_l1d.describe()

count    105974.000000
mean          5.235633
std           7.437533
min           1.000000
25%           1.000000
50%           3.000000
75%           6.000000
max         232.000000
Name: cnt_visit_l1d, dtype: float64

In [12]:
df_train = df.loc[(df.impression_time < "2018-12-07")  & (df.is_click.notnull()),:]
df_valid = df.loc[(df.impression_time >= "2018-12-07")  & (df.is_click.notnull()),:]
df_test = df.loc[df.is_click.isnull(),:]

In [13]:
df_train.shape, df_valid.shape, df_test.shape

((197093, 104), (40516, 104), (90675, 104))

In [17]:
input_cols = [x for x in df.columns.tolist() if x not in ['user_id',
'impression_id',
'impression_time',
'is_click',
'cnt_visit_category_1_5_l1d',
'cnt_visit_category_1_5_l3d',
'cnt_visit_category_1_5_l7d',
'cnt_visit_category_1_5_l14d',
'cnt_visit_category_1_5_l30d',
'cnt_visit_category_1_2_l1d',
'cnt_visit_category_1_2_l3d',
'cnt_visit_category_1_2_l7d',
'cnt_visit_category_1_2_l14d',
'cnt_visit_category_1_2_l30d']]

In [18]:
len(input_cols)

90

In [22]:
x_train, y_train = df_train[input_cols].values, df_train['is_click'].values
x_valid, y_valid = df_valid[input_cols].values, df_valid['is_click'].values
x_test, y_test = df_test[input_cols].values, df_test['is_click'].values

In [8]:
import sklearn
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

'import sklearn\nfrom xgboost import XGBClassifier\nfrom sklearn.metrics import classification_report\nfrom sklearn.model_selection import train_test_split'

In [ ]:
model = XGBClassifier()
xgb_clf = model

In [23]:
xgb_clf.fit(x_train, y_train, early_stopping_rounds=10,
            eval_metric="logloss", eval_set=[(x_valid, y_valid)])

[0]	validation_0-logloss:0.614175
Will train until validation_0-logloss hasn't improved in 10 rounds.
[1]	validation_0-logloss:0.549727
[2]	validation_0-logloss:0.49617
[3]	validation_0-logloss:0.451237
[4]	validation_0-logloss:0.41322
[5]	validation_0-logloss:0.380721
[6]	validation_0-logloss:0.352894
[7]	validation_0-logloss:0.329026
[8]	validation_0-logloss:0.308495
[9]	validation_0-logloss:0.290719
[10]	validation_0-logloss:0.275286
[11]	validation_0-logloss:0.261911
[12]	validation_0-logloss:0.250326
[13]	validation_0-logloss:0.24017
[14]	validation_0-logloss:0.231404
[15]	validation_0-logloss:0.223746
[16]	validation_0-logloss:0.217117
[17]	validation_0-logloss:0.211355
[18]	validation_0-logloss:0.206371
[19]	validation_0-logloss:0.202063
[20]	validation_0-logloss:0.198293
[21]	validation_0-logloss:0.195043
[22]	validation_0-logloss:0.192226
[23]	validation_0-logloss:0.189791
[24]	validation_0-logloss:0.187667
[25]	validation_0-logloss:0.185875
[26]	validation_0-logloss:0.18434
[

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [24]:
y_pred = xgb_clf.predict_proba(x_valid)[:,1]
y_pred_bin = xgb_clf.predict(x_valid)
predictions_bin = [round(value) for value in y_pred_bin]
predictions = y_pred

In [25]:
from sklearn import metrics

print(metrics.accuracy_score(y_valid, predictions_bin))
print(metrics.confusion_matrix(y_valid, predictions_bin))
print(metrics.roc_auc_score(y_valid, predictions))

0.9533270806594926
[[38623     0]
 [ 1891     2]]
0.7155474803305044


In [26]:
y_test_pred = xgb_clf.predict_proba(x_test)[:,1]

In [27]:
df_submission = pd.DataFrame()
df_submission['impression_id'] = df_test.impression_id
df_submission['is_click'] = y_test_pred

In [28]:
df_submission.to_csv(dirpath + '/Submissions/submission_xgboost_viewlog_unique_item.csv', index = False)

In [30]:
test.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,Hour,Weekday
0,a9e7126a585a69a32bc7414e9d0c0ada,2018-12-13 07:44:00,44754,127,latest,1,7,Thursday
1,caac14a5bf2ba283db7708bb34855760,2018-12-13 07:45:00,29656,44,latest,0,7,Thursday
2,13f10ba306a19ce7bec2f3cae507b698,2018-12-13 07:46:00,25234,296,latest,1,7,Thursday
3,39c4b4dc0e9701b55a0a4f072008fb3f,2018-12-13 07:47:00,22988,207,latest,1,7,Thursday
4,bf5a572cca75f5fc67f4b14e58b11d70,2018-12-13 07:48:00,35431,242,latest,1,7,Thursday


In [31]:
test.impression_id.nunique()

90675

In [4]:
average(2,3)

NameError: name 'average' is not defined